In [ ]:
import pandas as pd
import altair as alt
import numpy as np
import matplotlib.pyplot as plt

alt.data_transformers.disable_max_rows()


# dataDefault = pd.read_csv('https://raw.githubusercontent.com/byui-cse/cse450-course/master/data/housing.csv')
data = pd.read_csv('housingExpanded_3.csv')
data = data.drop(data.columns[[0]], axis=1)
data

,id,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,price,zipmean,zipstd,zipmin,zip25,zip50,zip75,zipmax,price15,waterfront15,year,month,dayOfMonth,PHSI,PHSI_1M_trend,PHSI_3M_trend,CCI
0,1565930130,20141104T000000,4,3.25,3760,4675,2.0,0,0,3,8,2740,1020,2007,0,98038,47.3862,-122.048,3280,4033,429900,364104,122461,110000,286500,340000,410500,1125000,528998.0000,0,2014,nov,4,4.7,2.0,4.3,100.44860
1,3279000420,20150115T000000,3,1.75,1460,7800,1.0,0,0,2,7,1040,420,1979,0,98023,47.3035,-122.382,1310,7865,233000,287363,109968,86500,224774,268750,316125,1275000,236913.3333,1,2015,jan,15,9.7,3.1,8.0,101.20860
2,194000575,20141014T000000,4,1.00,1340,5800,1.5,0,2,3,7,1340,0,1914,0,98116,47.5658,-122.389,1900,5800,455000,616679,243074,201500,464050,560000,685000,2050000,400813.3333,0,2014,oct,14,2.7,1.5,3.4,100.03910
3,2115510160,20141208T000000,3,1.75,1440,8050,1.0,0,0,3,8,1440,0,1985,0,98023,47.3187,-122.390,1790,7488,258950,287363,109968,86500,224774,268750,316125,1275000,336473.3333,1,2014,dec,8,7.6,2.9,6.4,100.90270
4,7522500005,20140815T000000,2,1.50,1780,4750,1.0,0,0,4,7,1080,700,1947,0,98117,47.6859,-122.395,1690,5962,555000,577525,189220,125000,450000,544500,658325,1370000,539966.8000,0,2014,aug,15,-2.0,1.1,3.2,99.46797
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19767,5272200045,20141113T000000,3,1.50,1000,6914,1.0,0,0,3,7,1000,0,1947,0,98125,47.7144,-122.319,1000,6947,378000,470202,209633,175000,357875,425000,513875,2050000,711983.3333,0,2014,nov,13,4.7,2.0,4.3,100.44860
19768,9578500790,20141111T000000,3,2.50,3087,5002,2.0,0,0,3,8,3087,0,2014,0,98023,47.2974,-122.349,2927,5183,399950,287363,109968,86500,224774,268750,316125,1275000,357209.0667,0,2014,nov,11,4.7,2.0,4.3,100.44860
19769,7202350480,20140930T000000,3,2.50,2120,4780,2.0,0,0,3,7,2120,0,2004,0,98053,47.6810,-122.032,1690,2650,575000,662387,252289,170000,479237,622500,784250,2140000,675462.5333,0,2014,sep,30,1.2,0.8,4.9,99.70537
19770,1723049033,20140620T000000,1,0.75,380,15000,1.0,0,0,3,5,380,0,1963,0,98168,47.4810,-122.323,1170,15000,245000,239567,66046,78000,199962,235000,283337,499950,443459.9333,0,2014,jun,20,-7.1,-0.9,0.9,99.36723


In [ ]:
# fix the data

data['waterfront'] = data['waterfront'].fillna(0)

# data = data[]

# get rid of outliers in bathrooms and bedrooms larger than 3 std away
from scipy import stats
data = data[(np.abs(stats.zscore(data['bathrooms'])) < 3)]
data = data[(np.abs(stats.zscore(data['bedrooms'])) < 3)]


# several have 0 bedrooms
data.drop(data.loc[data['bedrooms']==0].index)

data = data.reset_index(drop=True)

data

,id,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,price,zipmean,zipstd,zipmin,zip25,zip50,zip75,zipmax,price15,waterfront15
0,1565930130,20141104T000000,4,3.25,3760,4675,2.0,0,0,3,8,2740,1020,2007,0,98038,47.3862,-122.048,3280,4033,429900,364104,122461,110000,286500,340000,410500,1125000,528998.0000,0
1,3279000420,20150115T000000,3,1.75,1460,7800,1.0,0,0,2,7,1040,420,1979,0,98023,47.3035,-122.382,1310,7865,233000,287363,109968,86500,224774,268750,316125,1275000,236913.3333,1
2,194000575,20141014T000000,4,1.00,1340,5800,1.5,0,2,3,7,1340,0,1914,0,98116,47.5658,-122.389,1900,5800,455000,616679,243074,201500,464050,560000,685000,2050000,400813.3333,0
3,2115510160,20141208T000000,3,1.75,1440,8050,1.0,0,0,3,8,1440,0,1985,0,98023,47.3187,-122.390,1790,7488,258950,287363,109968,86500,224774,268750,316125,1275000,336473.3333,1
4,7522500005,20140815T000000,2,1.50,1780,4750,1.0,0,0,4,7,1080,700,1947,0,98117,47.6859,-122.395,1690,5962,555000,577525,189220,125000,450000,544500,658325,1370000,539966.8000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19490,5272200045,20141113T000000,3,1.50,1000,6914,1.0,0,0,3,7,1000,0,1947,0,98125,47.7144,-122.319,1000,6947,378000,470202,209633,175000,357875,425000,513875,2050000,711983.3333,0
19491,9578500790,20141111T000000,3,2.50,3087,5002,2.0,0,0,3,8,3087,0,2014,0,98023,47.2974,-122.349,2927,5183,399950,287363,109968,86500,224774,268750,316125,1275000,357209.0667,0
19492,7202350480,20140930T000000,3,2.50,2120,4780,2.0,0,0,3,7,2120,0,2004,0,98053,47.6810,-122.032,1690,2650,575000,662387,252289,170000,479237,622500,784250,2140000,675462.5333,0
19493,1723049033,20140620T000000,1,0.75,380,15000,1.0,0,0,3,5,380,0,1963,0,98168,47.4810,-122.323,1170,15000,245000,239567,66046,78000,199962,235000,283337,499950,443459.9333,0


In [ ]:
data

,Unnamed: 0,id,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,price,zipmean,zipstd,zipmin,zip25,zip50,zip75,zipmax,price15,waterfront15
0,0,1565930130,20141104T000000,4,3.25000,3760,4675,2.00000,0,0,3,8,2740,1020,2007,0,98038,47.38620,-122.04800,3280,4033,429900.00000,364104,122461,110000,286500,340000,410500,1125000,538991.33333,0
1,1,3279000420,20150115T000000,3,1.75000,1460,7800,1.00000,0,0,2,7,1040,420,1979,0,98023,47.30350,-122.38200,1310,7865,233000.00000,287363,109968,86500,224774,268750,316125,1275000,241646.66667,0
2,2,194000575,20141014T000000,4,1.00000,1340,5800,1.50000,0,2,3,7,1340,0,1914,0,98116,47.56580,-122.38900,1900,5800,455000.00000,616679,243074,201500,464050,560000,685000,2050000,383146.66667,0
3,3,2115510160,20141208T000000,3,1.75000,1440,8050,1.00000,0,0,3,8,1440,0,1985,0,98023,47.31870,-122.39000,1790,7488,258950.00000,287363,109968,86500,224774,268750,316125,1275000,334736.66667,1
4,4,7522500005,20140815T000000,2,1.50000,1780,4750,1.00000,0,0,4,7,1080,700,1947,0,98117,47.68590,-122.39500,1690,5962,555000.00000,577525,189220,125000,450000,544500,658325,1370000,536300.13333,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,19995,5272200045,20141113T000000,3,1.50000,1000,6914,1.00000,0,0,3,7,1000,0,1947,0,98125,47.71440,-122.31900,1000,6947,378000.00000,470202,209633,175000,357875,425000,513875,2050000,774883.33333,0
19996,19996,9578500790,20141111T000000,3,2.50000,3087,5002,2.00000,0,0,3,8,3087,0,2014,0,98023,47.29740,-122.34900,2927,5183,399950.00000,287363,109968,86500,224774,268750,316125,1275000,349872.40000,0
19997,19997,7202350480,20140930T000000,3,2.50000,2120,4780,2.00000,0,0,3,7,2120,0,2004,0,98053,47.68100,-122.03200,1690,2650,575000.00000,662387,252289,170000,479237,622500,784250,2140000,674929.20000,0
19998,19998,1723049033,20140620T000000,1,0.75000,380,15000,1.00000,0,0,3,5,380,0,1963,0,98168,47.48100,-122.32300,1170,15000,245000.00000,239567,66046,78000,199962,235000,283337,499950,431193.26667,0


In [ ]:
pd.set_option('display.float_format', lambda x: '%.5f' % x)
pd.set_option('display.max_rows', 100)
print(data['price'].describe())

count     20000.00000
mean     539436.71295
std      366433.43612
min       75000.00000
25%      322000.00000
50%      450000.00000
75%      641625.00000
max     7700000.00000
Name: price, dtype: float64


In [ ]:
data['view'].value_counts()

0    18030
2      889
3      471
1      311
4      299
Name: view, dtype: int64

In [ ]:
data['bedrooms'].value_counts()

3    9097
4    6301
2    2567
5    1412
6     216
1     179
Name: bedrooms, dtype: int64

# Map Plot - Price

In [ ]:
# install dependancies
!pip install --upgrade geopandas
import folium
import geopandas
import geopy

import folium.plugins
import branca
import branca.colormap as cm

In [ ]:
colormap = cm.LinearColormap(colors=['blue', 'red'], index=[75000,800000],vmin=75000,vmax=800000)

map1 = folium.Map(
    location=[47.5,-121],
    tiles='cartodbpositron',
    zoom_start=12,
)
data.apply(lambda row:folium.CircleMarker(
    location=[row["lat"], row["long"]],
    radius=1.5,
    color=colormap(row['price']),
    fill=True
    ).add_to(map1), axis=1)
map1

In [ ]:
# this dataset is too large to perform this geocoding. 

# from geopy.geocoders import Nominatim
# from geopy.extra.rate_limiter import RateLimiter

# geolocator = Nominatim(user_agent="my_request")
# geocode = RateLimiter(geolocator.reverse, min_delay_seconds=1.5)

# # data['coord'] = data['lat'] + ', ' + data['long']

# reverseTest1 = data[:100]
# # reverseTest

# moreDataTest = reverseTest1.apply(lambda x:geocode(str(x["lat"])+", "+str(x["long"]), addressdetails=True).raw['address'], axis=1)
# moreDataTest

In [ ]:
# # reverseTest['address'] = reverseTest['location'].str.split(',').str.get(1).str.strip()
# # reverseTest['address'] = reverseTest['location'].str.split(',').str.get(1)


# pd.set_option('display.max_colwidth', None)
# # reverseTest['location'][10]
# moreDataTestPD = pd.DataFrame.from_dict(moreDataTest)

# moreDataTestPD.columns = ['col1']

# moreDataTestPD = moreDataTestPD['col1'].apply(pd.Series).apply(pd.Series)

# moreDataTestPD
# # moreDataTest
# # reverseTest

In [ ]:
# moreDataTestPD.to_csv('housing_bonus.csv')

# files.download("housing_bonus.csv")

# Feature Engineering

In [ ]:
data[data['zipcode'] == 98023]

In [ ]:
# find additional statistics for each zip code and create new columns

zipcodesUnique = data['zipcode'].unique()

for i in zipcodesUnique:
  data.loc[data.zipcode == i, ['zipmean','zipstd', 'zipmin', 'zip25', 'zip50', 'zip75', 'zipmax']] = (
      data[data['zipcode'] == i].price.mean(),
      data[data['zipcode'] == i].price.std(),
      data[data['zipcode'] == i].price.min(),
      data[data['zipcode'] == i].price.quantile(0.25),
      data[data['zipcode'] == i].price.quantile(0.5),
      data[data['zipcode'] == i].price.quantile(0.75),
      data[data['zipcode'] == i].price.max()
    )

data[['zipmean','zipstd', 'zipmin', 'zip25', 'zip50', 'zip75', 'zipmax']] = data[['zipmean','zipstd', 'zipmin', 'zip25', 'zip50', 'zip75', 'zipmax']].astype(int)

In [ ]:
data.head(20)

,id,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,price,zipmean,zipstd,zipmin,zip25,zip50,zip75,zipmax,price15,waterfront15,year,month,dayOfMonth,PHSI,PHSI_1M_trend,PHSI_3M_trend,CCI,house_age
index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,1565930130,20141104T000000,4,3.25,3760,4675,2.0,0,0,3,8,2740,1020,2007,0,98038,47.3862,-122.048,3280,4033,429900.0,367155,133852,110000,286250,341000,411750,1350000,5.389913e+05,0,2014,nov,04,4.7,2.0,4.3,100.44860,7
1,3279000420,20150115T000000,3,1.75,1460,7800,1.0,0,0,2,7,1040,420,1979,0,98023,47.3035,-122.382,1310,7865,233000.0,287971,110628,86500,225000,269000,316500,1275000,2.416467e+05,0,2015,jan,15,9.7,3.1,8.0,101.20860,36
2,194000575,20141014T000000,4,1.00,1340,5800,1.5,0,2,3,7,1340,0,1914,0,98116,47.5658,-122.389,1900,5800,455000.0,623072,251853,201500,464762,566500,703500,2050000,3.831467e+05,0,2014,oct,14,2.7,1.5,3.4,100.03910,100
3,2115510160,20141208T000000,3,1.75,1440,8050,1.0,0,0,3,8,1440,0,1985,0,98023,47.3187,-122.390,1790,7488,258950.0,287971,110628,86500,225000,269000,316500,1275000,3.347367e+05,1,2014,dec,08,7.6,2.9,6.4,100.90270,29
4,7522500005,20140815T000000,2,1.50,1780,4750,1.0,0,0,4,7,1080,700,1947,0,98117,47.6859,-122.395,1690,5962,555000.0,578983,190652,125000,450000,545000,660000,1370000,5.363001e+05,0,2014,aug,15,-2.0,1.1,3.2,99.46797,67
5,3787000140,20140901T000000,3,2.25,1780,9969,1.0,0,0,3,8,1450,330,1985,0,98034,47.7286,-122.168,1950,7974,450000.0,517811,299868,90000,390375,446000,525000,3120000,6.788427e+05,0,2014,sep,01,1.2,0.8,4.9,99.70537,29
6,7852110690,20140522T000000,4,2.50,2980,8107,2.0,0,0,3,9,2980,0,2000,0,98065,47.5389,-121.876,2750,7760,622500.0,532883,186324,170000,425000,505000,622725,1998000,6.150767e+05,0,2014,may,22,-6.2,3.0,3.0,99.39791,14
7,5469700260,20140903T000000,4,2.25,2530,24700,2.0,0,0,3,7,2530,0,1974,0,98031,47.3939,-122.177,2650,24700,340000.0,302658,80403,147500,253000,289950,334975,800000,2.972933e+05,0,2014,sep,03,1.2,0.8,4.9,99.70537,40
8,8691410730,20150220T000000,4,2.50,3090,5600,2.0,0,0,3,9,3090,0,2005,0,98075,47.5970,-121.979,3080,5788,708000.0,793867,292131,406430,641000,739999,875000,3200000,8.326664e+05,0,2015,feb,20,12.7,3.0,9.0,101.21130,10


In [ ]:
from tqdm import tqdm
tqdm.pandas()

# functions to get the nearest neighbors and return the averaged values.

def dist(lat1, long1, lat2, long2):
    return np.abs((lat1-lat2)+(long1-long2))

def get_nearest_neighbors_mean(df, lat, long, neighbors, column, currentRow=None):
    distances = df.apply(lambda row: dist(lat, long, row['lat'], row['long']), axis=1)
    distances.columns = ['index', 'distance']
    # print(distances.nsmallest(neighbors).index.values.astype(int))

    if currentRow == None:
      nearest_list = df.iloc[distances.nsmallest(neighbors).index.values.astype(int)]
    else:
      nearest_list = df.iloc[distances.drop(index=currentRow).nsmallest(neighbors).index.values.astype(int)]
    # print(nearest_list[column].mean())
    return nearest_list[column].mean()

def get_nearest_neighbors_sum(df, lat, long, neighbors, column, currentRow=None):
    distances = df.apply(lambda row: dist(lat, long, row['lat'], row['long']), axis=1)
    distances.columns = ['index', 'distance']
    # print(distances.nsmallest(neighbors).index.values.astype(int))

    if currentRow == None:
      nearest_list = df.iloc[distances.nsmallest(neighbors).index.values.astype(int)]
    else:
      nearest_list = df.iloc[distances.drop(index=currentRow).nsmallest(neighbors).index.values.astype(int)]
    # print(nearest_list[column].mean())
    return nearest_list[column].sum()




# doing both metrics at the same time halfs the time needed to perform
def get_nearest_neighbors(df, lat, long, neighbors, currentRow=None):
    distances = df.apply(lambda row: dist(lat, long, row['lat'], row['long']), axis=1)
    distances.columns = ['index', 'distance']
    # print(distances.nsmallest(neighbors).index.values.astype(int))

    if currentRow == None:
      nearest_list = df.iloc[distances.nsmallest(neighbors).index.values.astype(int)]
    else:
      nearest_list = df.iloc[distances.drop(index=currentRow).nsmallest(neighbors).index.values.astype(int)]

    # print(nearest_list[columns[0]].mean())
    return nearest_list['price'].mean(), nearest_list['waterfront'].sum()








In [ ]:
data[['price15', 'waterfront15']] = data.progress_apply(
    lambda row: get_nearest_neighbors(data, row['lat'], row['long'], 15, row.name),
    axis=1,
    result_type='expand'
)

In [ ]:
data['price15'] = data.progress_apply(
    lambda row: get_nearest_neighbors_mean(data, row['lat'], row['long'], 15, 'price', row.name),
    axis=1
)


In [ ]:
data['waterfront15'] = data.progress_apply(
    lambda row: get_nearest_neighbors_sum(data, row['lat'], row['long'], 8, 'waterfront', row.name),
    axis=1
)


In [ ]:
from google.colab import files
data.to_csv('housingExpanded.csv')

files.download("housingExpanded.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# parse the date (year, month, day)

data['year'] = data['date'].str[:4]
data['month'] = data['date'].str[4:6]
data['dayOfMonth'] = data['date'].str[6:8]


# formatting

data['year'] = data['year'].astype(int)

data['month'] = data['month'].map({
  '01' : 'jan',
  '02' : 'feb',
  '03' : 'mar',
  '04' : 'apr',
  '05' : 'may',
  '06' : 'jun',
  '07' : 'jul',
  '08' : 'aug',
  '09' : 'sep',
  '10' : 'oct',
  '11' : 'nov',
  '12' : 'dec'})


In [ ]:
# National Pending Home Sales Index (PHSI)

phsi = pd.read_csv('housingPHSI_2014_2015.csv')

phsi['month'] = phsi['month'].astype(str)
phsi.dtypes

# dataTest = data[:50]

# dataTest.dtypes

data = data.reset_index().merge(phsi, on=['year','month']).set_index('index').sort_index()

In [ ]:
# Consumer Confidence Index (CCI)

cci = pd.read_csv('housingCCI_2014_2015.csv')

cci['month'] = cci['month'].astype(str)
cci.dtypes

# dataTest = data[:50]

# dataTest.dtypes

data = data.reset_index().merge(cci, on=['year','month']).set_index('index').sort_index()

In [ ]:
# house age 

data['house_age'] = (data['year'] - data['yr_built'])

In [ ]:
from google.colab import files
data.to_csv('housingExpanded (1).csv')

files.download("housingExpanded (1).csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Graphs

In [ ]:
alt.Chart(data).mark_bar().encode(
    alt.X("price:Q", bin=alt.Bin(extent=[0, 8000000], step=200000)),
    y='count()',
)

In [ ]:
alt.Chart(data).mark_bar().encode(
    x = 'year',
    y = 'price',
    color='price'
)

In [ ]:
alt.Chart(data).mark_bar().encode(
    x = 'month',
    y = 'price',
    color='price',
    column='year'
)

In [ ]:
alt.Chart(data).mark_bar().encode(
    x = 'dayOfMonth',
    y = 'price',
    color='price',
    column='year'
)

In [ ]:
alt.Chart(data).mark_bar().encode(
    alt.X("sqft_living", bin=alt.Bin(extent=[0, 8000], step=500)),
    y='price',
)

In [ ]:
alt.Chart(data).mark_bar().encode(
    alt.X("grade"),
    y='price',
)

In [ ]:
alt.Chart(data).mark_bar().encode(
    alt.X("zipcode"),
    y='price',
)

In [ ]:
alt.Chart(data).mark_bar().encode(
    alt.X("bathrooms"),
    y='price',
)

In [ ]:
alt.Chart(data).mark_bar().encode(
    alt.X("bedrooms"),
    y='price',
)

# Testing

In [ ]:
correlation = data.corr()
correlation['price'].sort_values(ascending=False)

price            1.000000
sqft_living      0.678762
grade            0.667737
zipmean          0.658267
zip75            0.654873
zip50            0.652363
zip25            0.644671
zipstd           0.595733
sqft_living15    0.594343
sqft_above       0.579434
zipmin           0.541928
zipmax           0.524304
bathrooms        0.490029
view             0.384928
price15          0.341685
lat              0.332729
bedrooms         0.303151
sqft_basement    0.296183
floors           0.255887
waterfront       0.235266
yr_renovated     0.121449
sqft_lot         0.083904
sqft_lot15       0.077108
condition        0.045763
yr_built         0.044368
long             0.023903
year             0.015765
CCI              0.000820
PHSI             0.000495
dayOfMonth      -0.010770
id              -0.015508
PHSI_1M_trend   -0.022529
PHSI_3M_trend   -0.030461
waterfront15    -0.043645
house_age       -0.044113
zipcode         -0.047291
Name: price, dtype: float64

In [ ]:
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_log_error
from sklearn.metrics import r2_score

In [ ]:
# X = dataDefault.drop(columns=['id', 'date', 'price'])
# y = dataDefault['price']


encodedData = pd.get_dummies(data, columns=[], drop_first=True)
X = encodedData.drop(columns=['id', 'date', 'zipcode', 'price', 'zip25', 'zip75', 'year', 'month', 'dayOfMonth',
                              'yr_renovated','sqft_above', 'CCI'])
y = encodedData['price']



def modelTrainingTest(randomnum):

  X_train, X_rem, y_train, y_rem = train_test_split(X, y, test_size=0.4, random_state=randomnum)

  X_test, X_dev, y_test, y_dev = train_test_split(X_rem, y_rem, test_size=0.5, random_state=randomnum)

  # print(X_train.shape), print(y_train.shape)
  # print(X_test.shape), print(y_test.shape)
  # print(X_dev.shape), print(y_dev.shape)

  params = {'n_estimators':900, 
            'max_depth':4, 
            'learning_rate':0.1, 
            'objective':'reg:squarederror', 
            'min_child_weight':3,
            'random_state':randomnum}

  model = XGBRegressor(   **params)
  model.fit(X_train, y_train)

  predictions = model.predict(X_test)

  resultRMSE = mean_squared_error(y_test, predictions, squared=False)
  resultRMSLE = np.sqrt(mean_squared_log_error(y_test, predictions))
  resultR2 = r2_score(y_test, predictions)

  # print('MSE: ' + str(resultMSE) + '\nMSLE: ' + str(resultMSLE) + '\nR2: ' + str(resultR2))


  # Final test with dev set
  devPredictions = model.predict(X_dev)

  devResultRMSE = mean_squared_error(y_dev, devPredictions, squared=False)
  devResultRMSLE = np.sqrt(mean_squared_log_error(y_dev, devPredictions))
  devResultR2 = r2_score(y_dev, devPredictions)
  # print('MSE: ' + str(devResultMSE) + '\nMSLE: ' + str(devResultMSLE) + '\nR2: ' + str(devResultR2))

  
  accuracy = [randomnum, resultRMSE, devResultRMSE, resultRMSLE, devResultRMSLE, resultR2, devResultR2]
  # print(accuracy)

  return accuracy, model, devPredictions, y_dev


# 50 seems like a good distribution
# 46/266 has a slightly lower r2 but also a lower MSE
# 355/907 almost gets 5 digit RMSE


accuracy, model, devPredictions, y_dev = modelTrainingTest(50)

print(accuracy)

[50, 103800.78645320675, 105629.08628953739, 0.16514411656785022, 0.16200187329136254, 0.8999421625408973, 0.9018680812612325]


In [ ]:
devPredictions1 = pd.DataFrame({'price_predicted':devPredictions, 'price_actual':y_dev})

devPredictions1.reset_index()

devPredictions_lowest = devPredictions1[devPredictions1['price_actual'].between(0, 400000)]
devPredictions_low = devPredictions1[devPredictions1['price_actual'].between(400001, 600000)]
devPredictions_mid = devPredictions1[devPredictions1['price_actual'].between(600001, 10000000)]

In [ ]:
# RMSE broken down into bins

RMSE_low = mean_squared_error(devPredictions_lowest['price_actual'], devPredictions_lowest['price_predicted'], squared=False)
print('RMSE(0-400000): ' + str(RMSE_low))
RMSE_lowest = mean_squared_error(devPredictions_low['price_actual'], devPredictions_low['price_predicted'], squared=False)
print('RMSE(400000-600000): ' + str(RMSE_lowest))
RMSE_mid = mean_squared_error(devPredictions_mid['price_actual'], devPredictions_mid['price_predicted'], squared=False)
print('RMSE(600000-...): ' + str(RMSE_mid))

RMSE(0-400000): 57439.10092457562
RMSE(400000-600000): 66606.39314550778
RMSE(600000-...): 172077.79250764177


In [ ]:
# run the model several times using different random numbers to check consistancy and average

import random
#Generate 5 random numbers between 10 and 30
randomlist = random.sample(range(1, 1000), 10)

accuracyList = []

for i in randomlist:
  accuracy = modelTrainingTest(i)[0]
  accuracyList.append(accuracy)

accuracyDf = pd.DataFrame(accuracyList, columns = ['seed', 'resultRMSE', 'devResultRMSE', 'resultRMSLE', 'devResultRMSLE', 'resultR2', 'devResultR2'])
accuracyDf

,seed,resultRMSE,devResultRMSE,resultRMSLE,devResultRMSLE,resultR2,devResultR2
0,987,104124.297721,104451.528422,0.162846,0.167678,0.907824,0.894453
1,916,105542.121238,103693.771021,0.163310,0.168972,0.900138,0.894169
2,120,103786.628734,102012.888143,0.165517,0.163401,0.898126,0.890407
3,238,107702.231286,100788.949489,0.164067,0.164599,0.888176,0.897712
4,653,103882.436062,105622.939833,0.162037,0.165748,0.901860,0.900623
5,61,106405.469969,105283.593051,0.169939,0.164150,0.895338,0.892407
6,584,101647.542441,106696.878301,0.160436,0.159578,0.902562,0.896659
7,907,109113.429351,100319.296897,0.168008,0.164346,0.896596,0.896841
8,27,108934.005407,104571.632846,0.163928,0.165467,0.892832,0.890059
9,743,102862.222948,105597.432587,0.163591,0.164753,0.900301,0.894140


# Final Test

In [ ]:
# proccess the raw data
# -------------------------

# zipcode stats

holdoutData = pd.read_csv('housing_holdout_test.csv')
holdoutData

temp = data[['zipcode', 'zipmean','zipstd', 'zipmin', 'zip25', 'zip50', 'zip75', 'zipmax']]
temp = temp.drop_duplicates()

holdoutData = holdoutData.reset_index().merge(temp, on=['zipcode']).set_index('index').sort_index()  # complicated   merge and keep index order



In [ ]:
# this process takes a long while
# neighbor waterfront property count
holdoutData[['price15', 'waterfront15']] = holdoutData.progress_apply(
    lambda row: get_nearest_neighbors(data, row['lat'], row['long'], 15),
    axis=1,
    result_type='expand'
)

100%|██████████| 1613/1613 [08:46<00:00,  3.06it/s]


In [ ]:
# process the date
holdoutData['year'] = holdoutData['date'].str[:4]
holdoutData['month'] = holdoutData['date'].str[4:6]
holdoutData['dayOfMonth'] = holdoutData['date'].str[6:8]

holdoutData['year'] = holdoutData['year'].astype(int)

holdoutData['month'] = holdoutData['month'].map({
  '01' : 'jan',
  '02' : 'feb',
  '03' : 'mar',
  '04' : 'apr',
  '05' : 'may',
  '06' : 'jun',
  '07' : 'jul',
  '08' : 'aug',
  '09' : 'sep',
  '10' : 'oct',
  '11' : 'nov',
  '12' : 'dec'})

In [ ]:
# add the PHSI values
phsi = pd.read_csv('housingPHSI_2014_2015.csv')

phsi['month'] = phsi['month'].astype(str)
holdoutData = holdoutData.reset_index().merge(phsi, on=['year','month']).set_index('index').sort_index()

In [ ]:
# add the CCI values

cci = pd.read_csv('housingCCI_2014_2015.csv')

cci['month'] = cci['month'].astype(str)
holdoutData = holdoutData.reset_index().merge(cci, on=['year','month']).set_index('index').sort_index()

In [ ]:
# add house age 

holdoutData['house_age'] = (holdoutData['year'] - holdoutData['yr_built'])

In [ ]:
# save the proccessed holdout dataset to avoid having to proccess it again
from google.colab import files
holdoutData.to_csv('housing_holdout_test_process_3.csv')

files.download("housing_holdout_test_process_3.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
finalModel = modelTrainingTest(50)[1]

# Save the Model as a pickle
import pickle
with open('final_model.pickle', 'wb') as handle:
    pickle.dump(finalModel, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
holdoutData = pd.read_csv('housing_holdout_test_process_3.csv')

X = holdoutData.drop(columns=['id', 'date', 'zipcode', 'zip25', 'zip75', 'year', 'month', 'dayOfMonth',
                              'yr_renovated','sqft_above', 'CCI'])

holdoutPredictions = finalModel.predict(X)
holdoutPredictions = pd.DataFrame(holdoutPredictions)

holdoutPredictions.to_csv('housing_holdout_predictions_3.csv')
files.download("housing_holdout_predictions_3.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>